In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn import metrics
import joblib
import datetime
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import time

n_days_lookahead = int(input('Please input the length of days lookahead in {5, 7, 15, 30, 45, 60, 90, 120}: '))

if(n_days_lookahead not in [5, 7, 15, 30, 45, 60, 90, 120]):
    print('Input does not meet requirements.')
    exit()

data_type = str(input('Please specify the coverage of the data {A - Manufacturer 1, B - Manufacturer 2, C - Manufacturer 1 & 2, D - Unbalanced}: '))

if(data_type not in ['A', 'B', 'C', 'D']):
    print('Input does not meet requirements.')
    exit()

dit_str = {'A': 'mc1', 'B': 'mc2', 'C': 'mc1_mc2', 'D': 'unbalanced'}

model_type = str(input('Please input the type of trained model to use {A - Manufacturer 1, B - Manufacturer 2, C - Manufacturer 1 & 2, D - Unbalanced}: '))

if(model_type not in ['A', 'B', 'C', 'D']):
    print('Input does not meet requirements.')
    exit()
    
def loadData():

    X = np.load('../data/' + dit_str[data_type] + '/' + str(n_days_lookahead) + '_days_lookahead/smart_test.npy',allow_pickle=True)
    y = np.load('../data/' + dit_str[data_type] + '/' + str(n_days_lookahead) + '_days_lookahead/test_labels.npy',allow_pickle=True)

    X = X.astype('float32')
    y = y.astype('float32')
    return X.reshape((len(X),30,-1)), y

def get_all_metrics(true, predicted, score):
    confusion_matrix = metrics.confusion_matrix(true, predicted)
    fpr_list, tpr_list, thersholds = roc_curve(true, score)
    roc_auc = auc(fpr_list, tpr_list)
    TP = confusion_matrix[0][0]
    FN = confusion_matrix[0][1]
    FP = confusion_matrix[1][0]
    TN =  confusion_matrix[1][1]
    precision_of_failed = TP / (TP + FP)
    precision_of_healthy = TN / (TN + FN)
    tpr = TP / (TP + FN)
    fpr = FP / (TN + FP)
    auc_score = roc_auc
    f1_score = 2*precision_of_failed*tpr / (precision_of_failed+tpr)
    print('precision of failed: ', precision_of_failed)
    print('precision of healthy: ', precision_of_healthy)
    print('tpr: ', tpr)
    print('fpr: ', fpr)
    print('auc: ', auc_score)
    print('f1-score: ', f1_score)
    print('roc curve: ')
    plt.plot(fpr_list, tpr_list)
    plt.xlim([-0.05, 1.05])  
    plt.ylim([-0.05, 1.05])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [ ]:
from sklearn.linear_model import LogisticRegression

rf = joblib.load('../trained_model/' + dit_str[model_type] + '/' + str(n_days_lookahead) + '_days_lookahead/rf.pkl')
knc = joblib.load('../trained_model/' + dit_str[model_type] + '/' + str(n_days_lookahead) + '_days_lookahead/knn.pkl')
lr = joblib.load('../trained_model/' + dit_str[model_type] + '/' + str(n_days_lookahead) + '_days_lookahead/lr.pkl')
dtc = joblib.load('../trained_model/' + dit_str[model_type] + '/' + str(n_days_lookahead) + '_days_lookahead/dt.pkl')
svc = joblib.load('../trained_model/' + dit_str[model_type] + '/' + str(n_days_lookahead) + '_days_lookahead/svm.pkl')

X_smart_test, y_smart_test = loadData()
print('------------------ RF ------------------')
y_pred = rf.predict(X_smart_test.reshape((len(X_smart_test),-1)))
y_pred_score = rf.predict_proba(X_smart_test.reshape(((len(X_smart_test),-1))))[:,1]
get_all_metrics(y_smart_test, y_pred, y_pred_score)

print('------------------ DT ------------------')
y_pred = dtc.predict(X_smart_test.reshape((len(X_smart_test),-1)))
y_pred_score = dtc.predict_proba(X_smart_test.reshape((len(X_smart_test),-1)))[:,1]
get_all_metrics(y_smart_test, y_pred, y_pred_score)

print('------------------ LR ------------------')
y_pred_score = lr.predict(X_smart_test.reshape((len(X_smart_test),-1)))
y_pred = np.zeros_like(y_pred_score)
for i in range(0,len(y_pred_score)):
    y_pred[i] = 0 if y_pred_score[i] < 0.5 else 1
get_all_metrics(y_smart_test, y_pred, y_pred_score)

print('------------------ SVM ------------------')
y_pred = svc.predict(X_smart_test.reshape((len(X_smart_test),-1)))
y_pred_score = svc.predict_proba(X_smart_test.reshape((len(X_smart_test),-1)))[:,1]
get_all_metrics(y_smart_test, y_pred, y_pred_score)

print('------------------ KNN ------------------')
y_pred = knc.predict(X_smart_test.reshape((len(X_smart_test),-1)))
y_pred_score = knc.predict_proba(X_smart_test.reshape(((len(X_smart_test),-1))))[:,1]
get_all_metrics(y_smart_test, y_pred, y_pred_score)


precision of failed:  0.8316377704542198
precision of healthy:  0.8566805362921868
tpr:  0.8618948618948619
fpr:  0.1744876744876745
auc:  0.910309449699114
f1-score:  0.846496025669073
728.0032634735107
precision of failed:  0.7837877030162413
precision of healthy:  0.7977784540474742
tpr:  0.8026433026433026
fpr:  0.22141372141372143
auc:  0.8372773567308761
f1-score:  0.7931034482758621
12.594938278198242
precision of failed:  0.6832557392839942
precision of healthy:  0.6576875259013676
tpr:  0.632016632016632
fpr:  0.292990792990793
auc:  0.713461664159615
f1-score:  0.6566381238910746
32.89294242858887
precision of failed:  0.7459262774877393
precision of healthy:  0.7175038477682943
tpr:  0.7001782001782002
fpr:  0.2384912384912385
auc:  0.7933419757744081
f1-score:  0.7223286097280737
88685.29891967773
precision of failed:  0.7720108695652174
precision of healthy:  0.8277668631303209
tpr:  0.8437778437778438
fpr:  0.2491832491832492
auc:  0.8692535360192029
f1-score:  0.80630055